In [2]:
!(cd pyflow; CXXFLAGS+=-stdlib=libc++ maturin develop --release)

package:   /Users/josh/josh/projects/tech/flow/tsunami/Cargo.toml
workspace: /Users/josh/josh/projects/tech/flow/Cargo.toml
🔗 Found pyo3 bindings
🐍 Found CPython 3.11 at /Users/josh/josh/projects/tech/flow/.venv/bin/python
💻 Using `MACOSX_DEPLOYMENT_TARGET=10.12` for x86_64-apple-darwin by default
package:   /Users/josh/josh/projects/tech/flow/tsunami/Cargo.toml
workspace: /Users/josh/josh/projects/tech/flow/Cargo.toml
   Compiling flow v0.1.0 (/Users/josh/josh/projects/tech/flow/flow)
   --> flow/src/bases.rs:382:9
    |
381 |         todo!();
    |         ------- any code following this expression is unreachable
382 |         values_theta_phi
    |         ^^^^^^^^^^^^^^^^ unreachable expression
    |
    = note: `#[warn(unreachable_code)]` on by default

   --> flow/src/bases.rs:518:9
    |
516 |         todo!();
    |         ------- any code following this expression is unreachable
517 |
518 |         b_mn
    |         ^^^^ unreachable expression

   --> flow/src/bases.rs:348:57

In [3]:
import traceback

import contextplot
import numpy as np
import tqdm

import pyflow

contextplot.set_defaults()

NS_PER_S = 1e9

num_cells = 80
kinematic_viscosity = 0.0005

# example = 'bump_1d'
# amplitude = 0.2
# width = 0.2
# t_max = 5
# solver = getattr(pyflow, example)(num_cells, num_z_cells, kinematic_viscosity, amplitude, width)

example = "bump_2d_spectral"
base_height = 5
amplitude = base_height * 0.2
solver = getattr(pyflow, example)(
    num_cells, kinematic_viscosity, base_height, amplitude
)

t_max = 3
speedup_factor = 0.15
frame_rate_hz = 30

t_max_ns = int(t_max * NS_PER_S)
dt_plot_ns = int(1 / frame_rate_hz * speedup_factor * NS_PER_S)


def frame_index(t_ns: int):
    return t_ns // dt_plot_ns


with contextplot.context_video(
    f"videos/{example}.mp4",
    1,
    size_inches=(2, 2),
    dpi=400,
    frame_rate_hz=frame_rate_hz,
    subplot_kw={"projection": "3d"},
) as cv:
    x_axis = solver.x_axis
    y_axis = solver.y_axis

    R = x_axis[-1] / (2 * np.pi)
    r = y_axis[-1] / (2 * np.pi)

    orig_x_grid, orig_y_grid = np.meshgrid(x_axis, y_axis, indexing='ij')
    theta_grid = orig_x_grid / x_axis[-1] * 2 * np.pi
    phi_grid = orig_y_grid / y_axis[-1] * 2 * np.pi
    out_uvec = np.stack(
        [-np.cos(theta_grid), np.sin(theta_grid), np.zeros_like(theta_grid)], axis=0
    )
    z_hat = np.array([0, 0, 1])[:, np.newaxis, np.newaxis]

    try:
        t_ns = 0
        bar = tqdm.tqdm(total=t_max_ns)
        while t_ns < t_max_ns:
            with cv.next_frame() as cp:
                fields = solver.fields
                height = fields.height
                velocity = fields.velocity

                pos_vec_grid = R * out_uvec + (r + height - base_height) * (
                    -np.cos(phi_grid)[np.newaxis, :, :] * out_uvec
                    + np.sin(phi_grid)[np.newaxis, :, :] * z_hat
                )
                output = cp.ax.plot_surface(
                    pos_vec_grid[0],
                    pos_vec_grid[1],
                    pos_vec_grid[2],
                    edgecolors="black",
                    linewidth=0.1,
                    alpha=0.7,
                    color="cyan",
                    rstride=1,
                    cstride=1,
                )
                cp.ax.set(
                    xlim=[-(R + 0.1 * r), (R + 0.1 * r)],
                    ylim=[-(R + 0.1 * r), (R + 0.1 * r)],
                    zlim=[-r * 1.1, r * 1.1],
                    aspect="equal",
                )
                cp.ax.set_axis_off()

            solver.integrate(dt_plot_ns / NS_PER_S)
            t_ns += dt_plot_ns
            if t_ns > t_max_ns:
                break
            _ = bar.update(dt_plot_ns)

    except KeyboardInterrupt:
        pass
    except BaseException:
        traceback.print_exc()
    finally:
        bar.close()

100%|██████████| 3000000000/3000000000 [09:54<00:00, 5043268.47it/s]
